**Imports and Reading Dataset**

In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from sklearn.preprocessing import MinMaxScaler
df = pd.read_excel("online_retail.xlsx", sheet_name="Year 2010-2011")
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom


**General Information About Dataset**

In [2]:
def information(df):
    print("###############################    Shape  ##################################")
    print(df.shape)
    print("###############################    Types  ##################################")
    print(df.dtypes)
    print("###############################    Head   ##################################")
    print(df.head())
    print("###############################    Tail   ##################################")
    print(df.tail())
    print("###############################    NA     ##################################")
    print(df.isnull().sum())
    print("############################### Quantiles ##################################")
    print(df.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

information(df)

###############################    Shape  ##################################
(541910, 8)
###############################    Types  ##################################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
###############################    Head   ##################################
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00   2.55     17850.00  United Kingdom
1  536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00   3.39     17850.00  United Kingdom
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00   2.75     17850.00  United Kingdom
3  536365  

**There is missing values. Description and Customer ID. We need to remove missing values**

In [3]:
df.dropna(inplace=True)
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

**The 'C' in the invoices shows the canceled transactions. Remove the canceled transactions from the dataset**

In [4]:
df[df["Invoice"].str.contains("C", na=False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.00,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.00,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.00,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.00,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.00,United Kingdom


In [5]:
df = df[~df["Invoice"].str.contains("C", na=False)]
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.00,13.02,180.42,1.00,2.00,6.00,12.00,80995.00
Price,397925.00,3.12,22.10,0.00,1.25,1.95,3.75,8142.75
Customer ID,397925.00,15294.31,1713.17,12346.00,13969.00,15159.00,16795.00,18287.00


**Finding Outliers and Replace with Thresholds**

In [6]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit 
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.00,13.02,180.42,1.00,2.00,6.00,12.00,80995.00
Price,397925.00,3.12,22.10,0.00,1.25,1.95,3.75,8142.75
Customer ID,397925.00,15294.31,1713.17,12346.00,13969.00,15159.00,16795.00,18287.00


In [8]:
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.00,11.83,25.53,1.00,2.00,6.00,12.00,298.50
Price,397925.00,2.89,3.23,0.00,1.25,1.95,3.75,37.06
Customer ID,397925.00,15294.31,1713.17,12346.00,13969.00,15159.00,16795.00,18287.00


**Creating 'TotalCost'**

In [9]:
df["TotalCost"] = df["Quantity"] * df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalCost
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.00,2010-12-01 08:26:00,2.55,17850.00,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6.00,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.00,2010-12-01 08:26:00,2.75,17850.00,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.00,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.00,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.00,11.83,25.53,1.00,2.00,6.00,12.00,298.50
Price,397925.00,2.89,3.23,0.00,1.25,1.95,3.75,37.06
Customer ID,397925.00,15294.31,1713.17,12346.00,13969.00,15159.00,16795.00,18287.00
TotalCost,397925.00,20.63,51.83,0.00,4.68,11.80,19.80,3268.57


**Customer Lifetime Value**

CLTV = ( Customer Value / Churn Rate ) x Profit Margin

Customer Value = Average Order Value * Purchase Frequency

Average Order Value = Total Price / Total transaction

Purchase Frequency = Total Transaction / Total Number of Customers

Churn Rate = 1 - Repeat Rate

Repeat Rate = Number of customers making multiple purchases / All customers

Profit Margin = Total Price * 0.10

Recency: How many weeks was the customer when he made the last purchase (Last Purchase Date - First Purchase Date)

T: The age of the customer. (Analysis Date - First Purchase Date)

Frequency. How often did the customer shop? (Must be greater than one)

Monetary: Average earnings per purchase

In [11]:
analysis_date = df["InvoiceDate"].max()+ dt.timedelta(days=2) #To calculate recency

In [24]:
cltv_df = df.groupby('Customer ID').agg(
    {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                     lambda InvoiceDate: (analysis_date - InvoiceDate.min()).days],
     'Invoice': lambda Invoice: Invoice.nunique(),
     'TotalCost': lambda TotalCost: TotalCost.sum(),
     'Country': lambda Country: Country.min()})
cltv_df.head()

InvoiceDate             Invoice TotalCost         Country
             <lambda_0> <lambda_1> <lambda>  <lambda>        <lambda>
Customer ID                                                          
12346.00              0        327        1    310.44  United Kingdom
12347.00            365        368        7   4310.00         Iceland
12348.00            282        359        4   1770.78         Finland
12349.00              0         20        1   1491.72           Italy
12350.00              0        311        1    331.46          Norway

**Rename Columns**

In [25]:
cltv_df.columns = ['Recency', 'T', 'Frequency', 'Monetary','Country']

In [26]:
cltv_df["Monetary"] = cltv_df["Monetary"] / cltv_df["Frequency"]
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4339.00,130.42,132.04,0.00,0.00,92.00,251.50,373.00
T,4339.00,224.26,117.92,2.00,114.00,250.00,328.00,375.00
Frequency,4339.00,4.27,7.71,1.00,1.00,2.00,5.00,210.00
Monetary,4339.00,364.01,367.16,0.00,176.77,288.18,421.99,6207.67


In [ ]:
cltv_df = cltv_df[(cltv_df['Frequency'] > 1)]
cltv_df = cltv_df[(cltv_df['Recency'] > 0)]
cltv_df["Recency"] = cltv_df["Recency"] / 7 

cltv_df["T"] = cltv_df["T"] / 7
cltv_df.head()

,Recency,T,Frequency,Monetary,Country
Customer ID,,,,,
12347.00,52.14,52.57,7,615.71,Iceland
12348.00,40.29,51.29,4,442.69,Finland
12352.00,37.14,42.57,8,219.54,Norway
12356.00,43.14,46.71,3,937.14,Portugal
12358.00,21.29,21.71,2,575.21,Austria


**Building of BG-NBD Model**

In [28]:
cltv_df_fr = cltv_df[(cltv_df['Country'].str.contains("France", na=False))]

In [30]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df_fr['Frequency'],
        cltv_df_fr['Recency'],
        cltv_df_fr['T'])

<lifetimes.BetaGeoFitter: fitted with 59 subjects, a: 0.02, alpha: 24.11, b: 1.41, r: 4.35>

**10 customers we expect to purchase the most in 6 months**

In [31]:
bgf.predict(4*6,
            cltv_df_fr['Frequency'],
            cltv_df_fr['Recency'],
            cltv_df_fr['T']).sort_values(ascending=False).head(10)

Customer ID
12682.00   10.94
12681.00    8.19
12437.00    7.47
12723.00    6.62
12685.00    6.49
12683.00    6.04
12583.00    5.95
12657.00    5.56
12536.00    5.22
12508.00    5.21
dtype: float64

In [32]:
cltv_df_fr["exp_purc_6_mon"] = bgf.predict(4 * 6,
                                               cltv_df_fr['Frequency'],
                                               cltv_df_fr['Recency'],
                                               cltv_df_fr['T'])
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,2783.00,29.05,15.88,0.14,15.14,30.14,43.07,53.29
T,2783.00,37.43,14.46,1.57,28.57,40.71,51.29,53.57
Frequency,2783.00,6.07,9.14,2.00,2.00,4.00,7.00,210.00
Monetary,2783.00,372.33,338.91,3.45,193.38,302.79,437.47,5724.30


**Building of GAMMA-GAMMA Model**

In [33]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv_df_fr['Frequency'], cltv_df_fr['Monetary'])

<lifetimes.GammaGammaFitter: fitted with 59 subjects, p: 3.71, q: 0.31, v: 3.66>

**10 customers we expected profit the most**

In [34]:
ggf.conditional_expected_average_profit(cltv_df_fr['Frequency'],
                                        cltv_df_fr['Monetary']).sort_values(ascending=False).head(10)

Customer ID
12731.00   1599.76
12536.00   1548.50
14277.00   1499.59
12678.00   1335.62
12700.00   1296.18
12714.00    978.92
12567.00    955.92
12656.00    842.30
12637.00    771.12
12689.00    689.78
dtype: float64

In [35]:
cltv_df_fr["exp_avg_profit"] = ggf.conditional_expected_average_profit(cltv_df_fr['Frequency'],
                                                                             cltv_df_fr['Monetary'])
cltv_df_fr.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,59.00,28.96,16.02,0.29,14.71,29.14,45.21,52.86
T,59.00,34.98,14.79,4.29,24.86,36.86,47.64,53.57
Frequency,59.00,6.12,5.38,2.00,3.00,4.00,7.00,31.00
Monetary,59.00,481.52,339.03,127.09,255.28,391.53,556.68,1574.66
exp_purc_6_mon,59.00,4.15,1.65,0.44,3.22,3.85,4.95,10.94
exp_avg_profit,59.00,505.33,351.14,142.14,271.04,418.71,583.75,1599.76


In [36]:
cltv_df_fr.sort_values("exp_avg_profit", ascending=False).head(10)

,Recency,T,Frequency,Monetary,Country,exp_purc_6_mon,exp_avg_profit
Customer ID,,,,,,,
12731.00,48.71,52.29,12,1574.66,France,5.11,1599.76
12536.00,2.14,8.57,3,1451.28,France,5.22,1548.50
14277.00,28.14,30.43,3,1405.40,France,3.19,1499.59
12678.00,44.43,50.71,12,1314.61,France,5.20,1335.62
12700.00,29.57,30.86,4,1235.00,France,3.61,1296.18
12714.00,40.71,42.29,5,941.77,France,3.35,978.92
12567.00,49.43,53.00,10,937.77,France,4.44,955.92
12656.00,30.86,33.57,5,810.23,France,3.85,842.30
12637.00,36.57,46.43,8,752.74,France,4.13,771.12


**Calculating CLTV with BG-NBD and GG model**

In [37]:
cltv_fr = ggf.customer_lifetime_value(bgf,
                                   cltv_df_fr['Frequency'],
                                   cltv_df_fr['Recency'],
                                   cltv_df_fr['T'],
                                   cltv_df_fr['Monetary'],
                                   time=6, 
                                   freq="W",
                                   discount_rate=0.01)
cltv_fr = cltv_fr.reset_index()
cltv_final_fr = cltv_df_fr.merge(cltv_fr, on="Customer ID", how="left")
cltv_final_fr.sort_values(by="clv", ascending=False).head(10)

,Customer ID,Recency,T,Frequency,Monetary,Country,exp_purc_6_mon,exp_avg_profit,clv
56,12731.00,48.71,52.29,12,1574.66,France,5.11,1599.76,8567.46
9,12536.00,2.14,8.57,3,1451.28,France,5.22,1548.50,8470.20
33,12678.00,44.43,50.71,12,1314.61,France,5.20,1335.62,7281.63
36,12681.00,50.43,52.71,22,626.71,France,8.19,632.22,5434.89
58,14277.00,28.14,30.43,3,1405.40,France,3.19,1499.59,5023.74
45,12700.00,29.57,30.86,4,1235.00,France,3.61,1296.18,4907.73
37,12682.00,52.43,53.14,31,396.39,France,10.94,398.91,4579.82
12,12567.00,49.43,53.00,10,937.77,France,4.44,955.92,4450.21
38,12683.00,51.57,52.43,15,565.10,France,6.04,572.45,3627.97
46,12714.00,40.71,42.29,5,941.77,France,3.35,978.92,3442.45


**Segmentation**

In [38]:
cltv_final_fr["Segment"] = pd.qcut(cltv_final_fr["clv"], 4, labels=["D", "C", "B", "A"])
cltv_final_fr.sort_values(by="clv", ascending=False)

,Customer ID,Recency,T,Frequency,Monetary,Country,exp_purc_6_mon,exp_avg_profit,clv,Segment
56,12731.00,48.71,52.29,12,1574.66,France,5.11,1599.76,8567.46,A
9,12536.00,2.14,8.57,3,1451.28,France,5.22,1548.50,8470.20,A
33,12678.00,44.43,50.71,12,1314.61,France,5.20,1335.62,7281.63,A
36,12681.00,50.43,52.71,22,626.71,France,8.19,632.22,5434.89,A
58,14277.00,28.14,30.43,3,1405.40,France,3.19,1499.59,5023.74,A
45,12700.00,29.57,30.86,4,1235.00,France,3.61,1296.18,4907.73,A
37,12682.00,52.43,53.14,31,396.39,France,10.94,398.91,4579.82,A
12,12567.00,49.43,53.00,10,937.77,France,4.44,955.92,4450.21,A
38,12683.00,51.57,52.43,15,565.10,France,6.04,572.45,3627.97,A
46,12714.00,40.71,42.29,5,941.77,France,3.35,978.92,3442.45,A


In [39]:
cltv_final_fr.groupby("Segment").agg({"count", "mean", "sum"})

Customer ID                    Recency                  T              Frequency            Monetary                 exp_purc_6_mon            exp_avg_profit                   clv                 
              count     mean       sum   count  mean    sum count  mean    sum     count  mean  sum    count   mean      sum          count mean   sum          count   mean      sum count    mean      sum
Segment                                                                                                                                                                                                     
D                15 12610.93 189164.00      15 32.57 488.57    15 43.24 648.57        15  4.27   64       15 216.89  3253.31             15 2.86 42.96             15 232.27  3484.00    15  632.55  9488.30
C                15 12649.60 189744.00      15 18.30 274.57    15 26.29 394.29        15  3.47   52       15 342.23  5133.48             15 3.78 56.66             15 367.76  5516.40    15 1347.63 20214.50
B                14 12617.00 176638.00      14 27.95 391.29    14 29.90 418.57        14  6.21   87       14 464.95  6509.30             14 4.60 64.34             14 488.80  6843.23    14 2257.56 31605.78
A                15 12758.13 191372.00      15 36.95 554.29    15 40.14 602.14        15 10.53  158       15 900.92 13513.74             15 5.39 80.85             15 931.40 13971.07    15 4796.16 71942.45

**Functionalization**

In [48]:
def create_cltv_p(dataframe, month=3): 
    dataframe.dropna(inplace=True)
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    dataframe["TotalCost"] = dataframe["Quantity"] * dataframe["Price"]
    analysis_date = df["InvoiceDate"].max()+ dt.timedelta(days=2)

    cltv_df = dataframe.groupby('Customer ID').agg(
        {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                         lambda InvoiceDate: (analysis_date - InvoiceDate.min()).days],
         'Invoice': lambda Invoice: Invoice.nunique(),
         'TotalCost': lambda TotalCost: TotalCost.sum()})

    cltv_df.columns = cltv_df.columns.droplevel(0)
    cltv_df.columns = ['Recency', 'T', 'Frequency', 'Monetary']
    cltv_df["Monetary"] = cltv_df["Monetary"] / cltv_df["Frequency"]
    cltv_df = cltv_df[(cltv_df['Frequency'] > 1)]
    cltv_df["Recency"] = cltv_df["Recency"] / 7
    cltv_df["T"] = cltv_df["T"] / 7

    
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['Frequency'],
            cltv_df['Recency'],
            cltv_df['T'])

    cltv_df["exp_purc_one_week"] = bgf.predict(1,
                                                  cltv_df['Frequency'],
                                                  cltv_df['Recency'],
                                                  cltv_df['T'])

    cltv_df["exp_purc_one_mon"] = bgf.predict(4,
                                                   cltv_df['Frequency'],
                                                   cltv_df['Recency'],
                                                   cltv_df['T'])

    cltv_df["expec_purc_3_mon"] = bgf.predict(12,
                                                   cltv_df['Frequency'],
                                                   cltv_df['Recency'],
                                                   cltv_df['T'])

    
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(cltv_df['Frequency'], cltv_df['Monetary'])
    cltv_df["exp_avg_profit"] = ggf.conditional_expected_average_profit(cltv_df['Frequency'],
                                                                                 cltv_df['Monetary'])

    
    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['Frequency'],
                                       cltv_df['Recency'],
                                       cltv_df['T'],
                                       cltv_df['Monetary'],
                                       time=month,  
                                       freq="W",  
                                       discount_rate=0.01)

    cltv = cltv.reset_index()
    cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
    cltv_final["Segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])

    return cltv_final

**Calculation 1-month and 12-month CLTV**

In [49]:
df_fr = df.copy()

In [50]:
df_fr = df_fr[(df_fr['Country'].str.contains("France", na=False))]
df_fr.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalCost
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24.00,2010-12-01 08:45:00,3.75,12583.00,France,90.00
27,536370,22727,ALARM CLOCK BAKELIKE RED,24.00,2010-12-01 08:45:00,3.75,12583.00,France,90.00
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12.00,2010-12-01 08:45:00,3.75,12583.00,France,45.00
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12.00,2010-12-01 08:45:00,0.85,12583.00,France,10.20
30,536370,21883,STARS GIFT TAPE,24.00,2010-12-01 08:45:00,0.65,12583.00,France,15.60


In [56]:
cltv_final_fr1 = create_cltv_p(df_fr,month = 1)
cltv_final_fr1

,Customer ID,Recency,T,Frequency,Monetary,exp_purc_one_week,exp_purc_one_mon,expec_purc_3_mon,exp_avg_profit,clv,Segment
0,12413.00,38.57,48.29,3,252.70,0.10,0.40,1.20,270.70,116.43,D
1,12437.00,47.00,47.43,18,275.08,0.31,1.25,3.74,278.15,373.28,B
2,12488.00,9.29,11.00,3,432.89,0.21,0.83,2.48,462.79,413.11,B
3,12490.00,46.29,47.29,10,541.79,0.20,0.80,2.40,552.43,476.48,B
4,12493.00,4.43,28.29,2,216.87,0.09,0.36,1.07,241.11,92.93,D
5,12494.00,49.86,52.43,7,178.01,0.15,0.59,1.77,183.41,116.55,D
6,12508.00,0.29,4.29,2,199.13,0.22,0.88,2.62,221.56,208.68,C
7,12523.00,44.57,47.57,11,162.71,0.21,0.85,2.56,165.84,152.26,D
8,12535.00,13.43,26.86,2,358.18,0.12,0.47,1.42,396.90,202.10,C
9,12536.00,2.14,8.57,3,1451.28,0.22,0.88,2.62,1548.47,1457.79,A


In [55]:
cltv_final_fr12 = create_cltv_p(df_fr,month = 12)
cltv_final_fr12

,Customer ID,Recency,T,Frequency,Monetary,exp_purc_one_week,exp_purc_one_mon,expec_purc_3_mon,exp_avg_profit,clv,Segment
0,12413.00,38.57,48.29,3,252.70,0.10,0.40,1.20,270.70,1313.28,D
1,12437.00,47.00,47.43,18,275.08,0.31,1.25,3.74,278.15,4220.18,B
2,12488.00,9.29,11.00,3,432.89,0.21,0.83,2.48,462.79,4637.33,B
3,12490.00,46.29,47.29,10,541.79,0.20,0.80,2.40,552.43,5384.20,B
4,12493.00,4.43,28.29,2,216.87,0.09,0.36,1.07,241.11,1045.11,D
5,12494.00,49.86,52.43,7,178.01,0.15,0.59,1.77,183.41,1316.90,D
6,12508.00,0.29,4.29,2,199.13,0.22,0.88,2.62,221.56,2335.81,C
7,12523.00,44.57,47.57,11,162.71,0.21,0.85,2.56,165.84,1720.73,D
8,12535.00,13.43,26.86,2,358.18,0.12,0.47,1.42,396.90,2272.55,C
9,12536.00,2.14,8.57,3,1451.28,0.22,0.88,2.62,1548.47,16355.40,A


In [57]:
cltv_final_fr1.groupby("Segment").agg({"count", "mean","median", "sum"})

Customer ID                             Recency                         T                     Frequency                   Monetary                        exp_purc_one_week                  exp_purc_one_mon                   expec_purc_3_mon                   exp_avg_profit                          clv                       
              count       sum     mean   median   count    sum  mean median count    sum  mean median     count  sum  mean median    count      sum   mean median             count  sum mean median            count   sum mean median            count   sum mean median          count      sum   mean median count      sum   mean median
Segment                                                                                                                                                                                                                                                                                                                                      
D                15 189164.00 12610.93 12604.00      15 488.57 32.57  37.29    15 648.57 43.24  47.57        15   64  4.27   4.00       15  3253.31 216.89 209.08                15 1.80 0.12   0.12               15  7.19 0.48   0.46               15 21.53 1.44   1.39             15  3483.93 232.26 220.23    15  1627.57 108.50 108.72
C                15 189744.00 12649.60 12669.00      15 274.57 18.30  15.00    15 394.29 26.29  25.57        15   52  3.47   3.00       15  5118.98 341.27 323.34                15 2.38 0.16   0.14               15  9.50 0.63   0.56               15 28.42 1.89   1.66             15  5501.31 366.75 358.50    15  3464.06 230.94 214.49
B                14 176638.00 12617.00 12648.50      14 391.29 27.95  26.71    14 418.57 29.90  30.21        14   87  6.21   4.50       14  6506.89 464.78 462.77                14 2.70 0.19   0.20               14 10.77 0.77   0.78               14 32.25 2.30   2.34             14  6840.59 488.61 490.21    14  5423.48 387.39 389.94
A                15 191372.00 12758.13 12681.00      15 554.29 36.95  40.71    15 602.14 40.14  46.43        15  158 10.53   8.00       15 13478.74 898.58 810.23                15 3.38 0.23   0.21               15 13.52 0.90   0.86               15 40.50 2.70   2.57             15 13935.37 929.02 842.29    15 12305.46 820.36 762.45

In [58]:
cltv_final_fr12.groupby("Segment").agg({"count", "mean","median", "sum"})

Customer ID                             Recency                         T                     Frequency                   Monetary                        exp_purc_one_week                  exp_purc_one_mon                   expec_purc_3_mon                   exp_avg_profit                          clv                          
              count       sum     mean   median   count    sum  mean median count    sum  mean median     count  sum  mean median    count      sum   mean median             count  sum mean median            count   sum mean median            count   sum mean median          count      sum   mean median count       sum    mean  median
Segment                                                                                                                                                                                                                                                                                                                                         
D                15 189164.00 12610.93 12604.00      15 488.57 32.57  37.29    15 648.57 43.24  47.57        15   64  4.27   4.00       15  3253.31 216.89 209.08                15 1.80 0.12   0.12               15  7.19 0.48   0.46               15 21.53 1.44   1.39             15  3483.93 232.26 220.23    15  18358.89 1223.93 1226.94
C                15 189744.00 12649.60 12669.00      15 274.57 18.30  15.00    15 394.29 26.29  25.57        15   52  3.47   3.00       15  5118.98 341.27 323.34                15 2.38 0.16   0.14               15  9.50 0.63   0.56               15 28.42 1.89   1.66             15  5501.31 366.75 358.50    15  38976.62 2598.44 2409.63
B                14 176638.00 12617.00 12648.50      14 391.29 27.95  26.71    14 418.57 29.90  30.21        14   87  6.21   4.50       14  6506.89 464.78 462.77                14 2.70 0.19   0.20               14 10.77 0.77   0.78               14 32.25 2.30   2.34             14  6840.59 488.61 490.21    14  61094.89 4363.92 4406.14
A                15 191372.00 12758.13 12681.00      15 554.29 36.95  40.71    15 602.14 40.14  46.43        15  158 10.53   8.00       15 13478.74 898.58 810.23                15 3.38 0.23   0.21               15 13.52 0.90   0.86               15 40.50 2.70   2.57             15 13935.37 929.02 842.29    15 138852.39 9256.83 8618.63